![](example_content/undatasio_example.png)

In [24]:
from dotenv import load_dotenv
from openai import OpenAI
import requests
import json
import os


# 初始化客户端
load_dotenv('.env')
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [22]:
def process_customer_inquiry(email_text: str) -> dict:
    """
    Extract structured data from a customer inquiry email using OpenAI.
    """
    try:
        prompt = f"""
        Based on the customer inquiry email below, extract the relevant details and return them as a JSON object. The JSON must have the following fields:
        1. customer_name: The full name of the customer.
        2. project_description: A brief summary of the project or question being asked.
        3. timeline: The stated timeline or deadline, if any.
        4. budget: The budget the customer has allocated, if mentioned.
        5. specific_requests: A list of any specific requests or questions from the customer.
        6. contact_email: The customer's email address.
        7. priority_level: High, Medium, or Low based on urgency indicated in the email.
        
        Customer inquiry email:
        ---
        {email_text}
        ---
        
        Provide the result in valid JSON format.
        """
        
        completion = client.chat.completions.create( 
           model="gpt-4o", 
           messages=[{"role": "user", "content": prompt}], 
           response_format={"type": "json_object"} 
        ) 
        
        return json.loads(completion.choices[0].message.content)
    except Exception as e:
        print(e)
        return f"{e}"

In [26]:
def send_to_crm(data: dict):
    """
    Sends structured customer inquiry data to a CRM as a lead.
    """
    # Prepare the payload for the CRM
    lead_payload = {
        "Name": data.get("customer_name", "Unknown"),
        "Company": "Unknown",  # Placeholder; update logic to extract company if needed
        "Email": data.get("contact_email", ""),
        "Phone": data.get("phone_number", None),  # Add support for phone if included
        "Description": data.get("project_description", "No description provided."),
        "Timeline": data.get("timeline", "Not specified"),
        "Budget": data.get("budget", "Not specified"),
        "Priority": data.get("priority_level", "Low"),
        "Specific Requests": ", ".join(data.get("specific_requests", []))
    }

    try:
        # Make an API call to the CRM
        
        # response = requests.post(
        #     CRM_API_URL,  # The URL for the CRM endpoint
        #     headers={"Authorization": f"Bearer {CRM_API_KEY}"},  # Add API authentication
        #     json=lead_payload  # Send the payload as JSON
        # )
        # response.raise_for_status()  # Raise exception for HTTP errors

        # Parse and return the CRM response
        # crm_response = response.json()
        return {
            "status": "success",
            # "crm_id": crm_response.get("id", "Unknown"),
            # "message": crm_response.get("message", "Lead created successfully.")
            "crm_id": "Unknown",
            "message": "Lead created successfully."
        }
    except requests.exceptions.RequestException as e:
        # Handle API request errors
        return {"status": "error", "message": str(e)}

In [27]:
email_text = """
Hi, I am Samantha Allyman and would like help improving our 
chatbot with agents for multi-step tasks. 
We need this completed within the next three months 
and have a budget of $15,000. 
Please let me know if this is possible.
Regards,
Samantha Allyman
(samantha.allyman@example.com)
"""

# FUNCTION 1: Process the email text
inquiry_data = process_customer_inquiry(email_text)

print(inquiry_data)
# FUNCTION 2: Now pass the structured data to the CRM function
crm_response = send_to_crm(inquiry_data)

{'customer_name': 'Samantha Allyman', 'project_description': 'Improving our chatbot with agents for multi-step tasks.', 'timeline': 'Within the next three months', 'budget': '$15,000', 'specific_requests': ['Please let me know if this is possible.'], 'contact_email': 'samantha.allyman@example.com', 'priority_level': 'Medium'}
